In [62]:
# Install dependencies
%pip install -q langchain langchain-core langchain-community google-genai python-dotenv langchain_google_genai


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [63]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
import os
from dotenv import load_dotenv

In [64]:
load_dotenv()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=os.getenv("GOOGLE_API_KEY"),
    max_output_tokens=1024,
)

In [65]:
# Test LLM configuraiion
llm.invoke("Hello")

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--54783a0b-f274-47f8-82eb-c6e85ec06413-0', usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})

### Create simple store for chat histories

In [66]:
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
        
    return store[session_id]

In [67]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [68]:
chain = prompt | llm

In [69]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [70]:
session_one_id = "s1"
session_two_id = "s2"
response = chain_with_history.invoke(
    {
        "input": "Hello"
    },
    config={"configurable": {"session_id": session_one_id}},
)

print(f"Session {session_one_id} AI:", response.content)

response = chain_with_history.invoke(
    {
        "input": "What is the last message"
    },
    config={"configurable": {"session_id": session_one_id}},
)

print(f"Session {session_one_id} AI:", response.content)

response = chain_with_history.invoke(
    {
        "input": "What is the last message"
    },
    config={"configurable": {"session_id": session_two_id}},
)

print(f"Session {session_two_id} AI:", response.content)


print("\nConversation History:")
for message in store[session_one_id].messages:
    print(f"{message.type}: {message.content}")


Session s1 AI: Hello! How can I help you today?
Session s1 AI: The last message I sent was "Hello! How can I help you today?".
Session s2 AI: I'm sorry, I don't have access to past conversations or the ability to remember previous interactions. Therefore, I don't know what the "last message" is.

Conversation History:
human: Hello
ai: Hello! How can I help you today?
human: What is the last message
ai: The last message I sent was "Hello! How can I help you today?".
